In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('/mnt/d/image.png', cv2.IMREAD_GRAYSCALE)

def compute_skew(image):
    image = cv2.bitwise_not(image)
    height, width = image.shape

    edges = cv2.Canny(image, 150, 200, 3, 5)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=width / 2.0, maxLineGap=20)
    angle = 0.0
    nlines = lines.size
    for x1, y1, x2, y2 in lines[0]:
        angle += np.arctan2(y2 - y1, x2 - x1)
    return angle / nlines


def deskew(image, angle):
    image = cv2.bitwise_not(image)
    non_zero_pixels = cv2.findNonZero(image)
    center, wh, theta = cv2.minAreaRect(non_zero_pixels)

    root_mat = cv2.getRotationMatrix2D(center, angle, 1)
    rows, cols = image.shape
    rotated = cv2.warpAffine(image, root_mat, (cols, rows), flags=cv2.INTER_CUBIC)
    cv2.imwrite('/mnt/d/rotated.jpg',rotated)
    return cv2.getRectSubPix(rotated, (cols, rows), center)


deskewed_image = deskew(img.copy(), compute_skew(img))

In [3]:
cv2.__version__

'4.0.0'

In [13]:
cv2.imwrite('/mnt/d/s.jpg',deskewed_image)

True

In [12]:
def compute_skew(image):
    # image = cv2.bitwise_not(image)
    height, width = image.shape

    edges = cv2.Canny(image, 150, 200, 3, 5)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=width / 2.0, maxLineGap=20)
    angle = 0.0
    number_of_line = lines.size
    for x1, y1, x2, y2 in lines[0]:
        if x1 != x2:
            angle += np.arctan(y2 - y1 / x2 - x1)
    print(angle , number_of_line)
    return angle / number_of_line


def deskew(image, angle):
    angle = np.math.degrees(angle)
    print(angle)
    # image = cv2.bitwise_not(image)
    non_zero_pixels = cv2.findNonZero(image)
    center, wh, theta = cv2.minAreaRect(non_zero_pixels)

    root_mat = cv2.getRotationMatrix2D(center, angle, 1)
    rows, cols = image.shape
    rotated = cv2.warpAffine(image, root_mat, (cols, rows), flags=cv2.INTER_CUBIC)

    return cv2.getRectSubPix(rotated, (cols, rows), center)

In [13]:
deskewed_image = deskew(img.copy(), compute_skew(img))

1.5702509284135058 48
1.8743489786373548


In [31]:
img

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [16]:
compute_skew(img)

1.5702509284135058 48


0.0327135610086147

In [19]:
cv2.imwrite('/mnt/d/DjangoProjectImage/mysite/rota.jpg',deskew(img, 0.0327135610086147))

1.8743489786373548


True

In [32]:
def deskew(image):
    # load the image from disk
    cv2.imwrite('Originalimage.jpg', image)
    # convert the image to grayscale and flip the foreground and background to ensure foreground is now "white" and the background is "black"
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(image)

    # threshold the image, setting all foreground pixels to 255 and all background pixels to 0
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
    # compute a rotated bounding box that contains all coordinates
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    print(angle)
    # the `cv2.minAreaRect` function returns values in the  range [-90, 0); as the rectangle rotates clockwise the
    # returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle
    if angle < -45:
        angle = -(90 + angle)

    # otherwise, just take the inverse of the angle to make it positive
    else:
        angle = -angle

    # rotate the image to deskew it
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    # draw the correction angle on the image so we can validate it
    # cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # show the output image
    print("[INFO] angle: {:.3f}".format(angle))
    cv2.imwrite('/mnt/d/DjangoProjectImage/mysite/rotated1.jpg', rotated)
    return rotated


In [34]:
img=cv2.imread('/mnt/d/DjangoProjectImage/mysite/rotated.jpg',0)
deskew(img)

0.0
[INFO] angle: -0.000


array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [62]:
def deskew(im, max_skew=10):
    height, width,_ = im.shape
    im_gs = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gs = cv2.fastNlMeansDenoising(im_gs, h=3)
    im_bw = cv2.threshold(im_gs, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    lines = cv2.HoughLinesP(im_bw, 1, np.pi / 180, 200, minLineLength=width / 12, maxLineGap=width / 150)
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angles.append(np.arctan2(y2 - y1, x2 - x1))
    landscape = np.sum([abs(angle) > np.pi / 4 for angle in angles]) > len(angles) / 2
    if landscape:
        angles = [
            angle
            for angle in angles
            if np.deg2rad(90 - max_skew) < abs(angle) < np.deg2rad(90 + max_skew)
        ]
    else:
        angles = [angle for angle in angles if abs(angle) < np.deg2rad(max_skew)]
    if len(angles) < 5:
        return im
    angle_deg = np.rad2deg(np.median(angles))
    if landscape:
        if angle_deg < 0:
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            angle_deg += 90
        elif angle_deg > 0:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
            angle_deg -= 90
    M = cv2.getRotationMatrix2D((width / 2, height / 2), angle_deg, 1)
    im = cv2.warpAffine(im, M, (width, height), borderMode=cv2.BORDER_REPLICATE)
    cv2.imwrite('/mnt/d/imageqq.png',im)
    return im


In [63]:
deskew(cv2.imread('/mnt/d/text-image-python/Al-Rawabi/AlRawabi/2019/10/10/Precisio/Input/'))


array([[[ 80,  80,  80],
        [ 80,  80,  80],
        [ 80,  80,  80],
        ...,
        [230, 230, 230],
        [230, 230, 230],
        [230, 230, 230]],

       [[ 80,  80,  80],
        [ 80,  80,  80],
        [ 80,  80,  80],
        ...,
        [230, 230, 230],
        [230, 230, 230],
        [230, 230, 230]],

       [[ 80,  80,  80],
        [ 80,  80,  80],
        [ 80,  80,  80],
        ...,
        [230, 230, 230],
        [230, 230, 230],
        [230, 230, 230]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [215, 131,  24],
        [214, 164, 101],
        [215, 215, 215]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [215, 131,  24],
        [214, 162,  95],
        [215, 215, 215]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [215, 131,  24],
        [214, 162,  95],
        [215, 215, 215]]

In [ ]:
# load the image from disk
#image = cv2.imread(imgpath)
# convert the image to grayscale and flip the foreground and background to ensure foreground is now "white" and the background is "black"
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bitwise_not(image)

# threshold the image, setting all foreground pixels to 255 and all background pixels to 0
thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
# compute a rotated bounding box that contains all coordinates
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

# the `cv2.minAreaRect` function returns values in the  range [-90, 0); as the rectangle rotates clockwise the
# returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle
if angle < -45:
    angle = -(90 + angle)

# otherwise, just take the inverse of the angle to make it positive
else:
    angle = -angle

# rotate the image to deskew it
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(image, M, (w, h),flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
# draw the correction angle on the image so we can validate it
# cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

# show the output image
print("[INFO] angle: {:.3f}".format(angle))
cv2.imwrite('rotated.jpg',rotated)
return rotated
